In [1]:
!pip install pyspark
!pip install findspark

# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Check Spark Session Information
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=f6e35bb6bf9fac4bf9c0fdf098f01708f2a401f80b7221fdbe4eebb4c07658c9
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

# Prepare training and test data.
data = spark.read.format("libsvm")\
    .load("/content/sample_linear_regression_data.txt")
train, test = data.randomSplit([0.9, 0.1], seed=12345)

lr = LinearRegression(maxIter=10)

# We use a ParamGridBuilder to construct a grid of parameters to search over.
# TrainValidationSplit will try all combinations of values and determine best model using
# the evaluator.
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(train)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
model.transform(test)\
    .select("features", "label", "prediction")\
    .show()

+--------------------+--------------------+--------------------+
|            features|               label|          prediction|
+--------------------+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -17.026492264209548| -1.6552077975758144|
|(10,[0,1,2,3,4,5,...|  -16.71909683360509|-0.06173284202439916|
|(10,[0,1,2,3,4,5,...| -15.375857723312297|  0.6949088125477225|
|(10,[0,1,2,3,4,5,...| -13.772441561702871|  3.9104769954029184|
|(10,[0,1,2,3,4,5,...| -13.039928064104615|  0.3433433960676067|
|(10,[0,1,2,3,4,5,...|   -9.42898793151394| -4.0115565332806575|
|(10,[0,1,2,3,4,5,...|    -9.2679651250406|-0.05502173816316347|
|(10,[0,1,2,3,4,5,...|  -9.173693798406978|-0.28813191486076367|
|(10,[0,1,2,3,4,5,...| -7.1500991588127265|   3.418347141475304|
|(10,[0,1,2,3,4,5,...|  -6.930603551528371|0.030343256659171147|
|(10,[0,1,2,3,4,5,...|  -6.456944198081549|-0.45788602578523113|
|(10,[0,1,2,3,4,5,...| -3.2843694575334834| -0.7807226371771443|
|(10,[0,1,2,3,4,5,...|   